In [ ]:
from dotenv import load_dotenv
from pathlib import Path
import sys


sys.path.append(Path("..").resolve().as_posix())
_ = load_dotenv()

In [ ]:
target_images_path = Path("fashion_twitter_raw/images").resolve()
target_annotations_path = Path("fashion_twitter_raw/fashion_twitter_converted.jsonl")

## Convert annotations

In [ ]:
from pydantic import BaseModel, HttpUrl
from typing import List, Optional
import requests
import json

In [ ]:
raw_tweets_path = Path("fashion_twitter_raw/fashion_twitter.jsonl")


class Recommendations(BaseModel):
    problems: List[str]
    fixes: List[str]
    positives: Optional[List[str]] = []
    advice: str


class Tweet(BaseModel):
    id: str
    image_url: HttpUrl
    description: str
    recommendations: Recommendations


with raw_tweets_path.open("r") as f:
    tweets = [Tweet.model_validate_json(line) for line in f.readlines()]

In [ ]:
target_images_path.mkdir(exist_ok=True, parents=True)

# Download each image
for tweet in tweets:
    response = requests.get(tweet.image_url)
    if response.status_code == 200:
        with target_images_path.joinpath(f"{tweet.id}.jpg").open("wb") as f:
            f.write(response.content)
        print(f"Downloaded {tweet.id}.jpg")
    else:
        print(f"Failed to download image from {tweet.image_url}")

print("All images downloaded.")

In [ ]:
with target_annotations_path.open("w") as f:
    for tweet in tweets:
        json_dict = {
            "image": target_images_path.joinpath(f"{tweet.id}.jpg").as_posix(),
            "json": tweet.model_dump_json(include=["description", "recommendations"]),
        }
        f.write(f"{json.dumps(json_dict)}\n")

## Import dataset

In [ ]:
from training_toolkit import ImageJSONImporter

In [ ]:
image_importer = ImageJSONImporter()
dataset = image_importer("fashion_twitter", target_images_path.as_posix(), target_annotations_path.as_posix())
dataset.save_to_disk("fashion_twitter")

In [ ]:
from training_toolkit import build_trainer, paligemma_image_preset, image_json_preset

In [ ]:
trainer = build_trainer(**image_json_preset.with_path("fashion_twitter").as_kwargs(), **paligemma_image_preset.as_kwargs())

In [ ]:
trainer.train()